### Loading API Keys
Set the environment `OPENAI_API_KEY in a .env file which we will be loading using dotenv.

In [ ]:
import dotenv
dotenv.load_dotenv("../.env", override=True)

### Setting up the model
We will start by initializing the language model. The model can be easily swapped here for API or local models

In [36]:
import dspy

lm = dspy.OpenAI(model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

### DSPy fundamentals

In [32]:
question = "Who was the first person to walk on the moon?"

qa = dspy.Predict('question -> piglatin')
response = qa(question=question)

print(response.piglatin)

Question: Who was the first person to walk on the moon?
Piglatin: HoWay asway ethay irstfay ersonpay otay alkway onway ethay oonmay?


In [55]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

predictor = dspy.ChainOfThought(BasicQA)
result = predictor(
    question=("What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?")
)
print(result)

Prediction(
    rationale='produce the answer. We can recall that Robert Irvine, the chef and restaurateur featured in Restaurant: Impossible, is from a specific country.',
    answer='British'
)


In [39]:
lm.inspect_history()





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: often between 1 and 5 words

---

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Reasoning: Let's think step by step in order to produce the answer. We can recall that Robert Irvine, the chef and restaurateur featured in Restaurant: Impossible, is from a specific country.
Answer: British





#### Datasets/Examples

In [52]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/Users/asleshapokhrel/Library/Caches/pypoetry/virtualenvs/tmls-llm-selection-m0VYMRV3-py3.12/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

In [63]:
train_example = trainset[0]
train_example

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

### RAG

In [59]:
retrieve = dspy.Retrieve(k=3)
topK_passages = retrieve(train_example.question).passages

print(f"Top {retrieve.k} passages for question: {train_example.question} \n", '-' * 30, '\n')

for idx, passage in enumerate(topK_passages):
    print(f'{idx+1}]', passage, '\n')

Top 3 passages for question: At My Window was released by which American singer-songwriter? 
 ------------------------------ 

1] At My Window (album) | At My Window is an album released by Folk/country singer-songwriter Townes Van Zandt in 1987. This was Van Zandt's first studio album in the nine years that followed 1978's "Flyin' Shoes", and his only studio album recorded in the 1980s. Although the songwriter had become less prolific, this release showed that the quality of his material remained high. 

2] Little Window | Little Window is the debut album of American singer-songwriter Baby Dee. The album was released in 2002 on the Durtro label. It was produced, composed, and performed entirely by Dee. 

3] Windows and Walls | Windows and Walls is the eighth album by American singer-songwriter Dan Fogelberg, released in 1984 (see 1984 in music). The first single, "The Language of Love", reached 13 on the U.S. "Billboard" Hot 100 chart. Although the follow-up, "Believe in Me", missed t

In [62]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

rag = RAG()
rag(train_example.question)

Prediction(
    context=['At My Window (album) | At My Window is an album released by Folk/country singer-songwriter Townes Van Zandt in 1987. This was Van Zandt\'s first studio album in the nine years that followed 1978\'s "Flyin\' Shoes", and his only studio album recorded in the 1980s. Although the songwriter had become less prolific, this release showed that the quality of his material remained high.', 'Little Window | Little Window is the debut album of American singer-songwriter Baby Dee. The album was released in 2002 on the Durtro label. It was produced, composed, and performed entirely by Dee.', 'Windows and Walls | Windows and Walls is the eighth album by American singer-songwriter Dan Fogelberg, released in 1984 (see 1984 in music). The first single, "The Language of Love", reached 13 on the U.S. "Billboard" Hot 100 chart. Although the follow-up, "Believe in Me", missed the Top 40 of the pop chart, peaking at No. 48, it became the singer\'s fourth No. 1 song on the "Billboar